<a href="https://colab.research.google.com/github/sudeepmenon17/PytorchforNN/blob/main/pt_to_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install google-auth
from google.colab import auth
from google.colab import drive

drive.mount('/content/drive')

# !pip install torchvision
!pip install onnx

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import torch
import torch.nn as nn
import torch.onnx
from torch.autograd import Variable

# Define the PyTorch model class
class Model(nn.Module):
    def __init__(self, in_features=5, h1=61, h2=60, out_features=1):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.out(x)
        return x

In [17]:
# Load the saved PyTorch model
model = Model()
model.load_state_dict(torch.load('/content/drive/MyDrive/komegaSST_data/airfoil_model.pt'))
model.eval()

<ipython-input-17-fb4db81e98fc>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/komegaSST_data/airfoil_model.pt'))


Model(
  (fc1): Linear(in_features=5, out_features=61, bias=True)
  (fc2): Linear(in_features=61, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [18]:
# Define file paths
onnx_file_path = "/content/drive/MyDrive/komegaSST_data/airfoil_model.onnx"
dummy_input = torch.randn(1,5)

# Step 1: Export PyTorch model to ONNX format
def export_pytorch_to_onnx(model, dummy_input, onnx_file_path):
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output']
    )

In [19]:
# Perform the conversion
export_pytorch_to_onnx(model, dummy_input, onnx_file_path)


In [21]:
!pip install onnxruntime
import numpy as np
import onnxruntime
import torch

# Define a function to check the consistency of the two models
def compare_pytorch_onnx(pytorch_model, onnx_model_path, dummy_input):
    # Ensure PyTorch model is in evaluation mode
    pytorch_model.eval()

    # Get the PyTorch model's output
    with torch.no_grad():
        pytorch_output = pytorch_model(dummy_input)

    # Load ONNX model using ONNX Runtime
    session = onnxruntime.InferenceSession(onnx_model_path)

    # Prepare input for the ONNX model
    dummy_input_numpy = dummy_input.numpy()  # Convert PyTorch tensor to NumPy array
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Get the ONNX model's output
    onnx_output = session.run([output_name], {input_name: dummy_input_numpy})[0]

    # Compare outputs
    print("PyTorch Model Output:", pytorch_output.numpy())
    print("ONNX Model Output:", onnx_output)
    print("Difference:", np.abs(pytorch_output.numpy() - onnx_output))
    print("Are the outputs close?:", np.allclose(pytorch_output.numpy(), onnx_output, atol=1e-5))

# Define a dummy input
dummy_input = torch.randn(1, 5)

# Compare PyTorch and ONNX outputs
compare_pytorch_onnx(model, onnx_file_path, dummy_input)


PyTorch Model Output: [[0.98726714]]
ONNX Model Output: [[0.98726726]]
Difference: [[1.1920929e-07]]
Are the outputs close?: True


In [22]:

# Define a dummy input
dummy_input = torch.randn(1, 5)

# Compare PyTorch and ONNX outputs
compare_pytorch_onnx(model, onnx_file_path, dummy_input)

PyTorch Model Output: [[0.9473942]]
ONNX Model Output: [[0.947394]]
Difference: [[1.7881393e-07]]
Are the outputs close?: True


In [23]:

# Define a dummy input
dummy_input = torch.randn(1, 5)

# Compare PyTorch and ONNX outputs
compare_pytorch_onnx(model, onnx_file_path, dummy_input)

PyTorch Model Output: [[1.0613629]]
ONNX Model Output: [[1.0613629]]
Difference: [[0.]]
Are the outputs close?: True
